In [4]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import datetime as dt
from lib import FiData as data
from sklearn.model_selection  import train_test_split
from __future__ import division
from __future__ import print_function  
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

start_date = dt.date(2005,5,1)
end_date = dt.date(2016,12,31)
rolldays = 40
factor_nums = 16
label_nums = 5

# X,Y = data.get_data(data.Idx_hs300,start_date,end_date,norm=False)
# X = data.data_roll(X,rolldays)
# Y = Y[rolldays:]
# Y = data.get_label5(Y) ## 根据label nums 替换

# np.save("data/"+data.Idx_hs300+"_X.npy",X)
# np.save('data/'+data.Idx_hs300+'_Y.npy',Y)

X = np.load('data/'+data.Idx_hs300+'_X.npy')
Y = np.load('data/'+data.Idx_hs300+'_Y.npy')

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

sess = tf.Session()
xs_train = x_train
ys_train = sess.run(tf.one_hot(y_train,label_nums))
xs_test = x_test
ys_test = sess.run(tf.one_hot(y_test,label_nums))
sess.close()
print (xs_train.shape, ys_train.shape,xs_test.shape,ys_test.shape)
# ys = sess.run(tf.reshape(ys,[len(ys),label_nums]))
# ys_test = sess.run(tf.reshape(ys_test,[len(ys_test),label_nums]))
# x_train[0:1],y_train[0:1]

(2212, 640) (2212, 5) (553, 640) (553, 5)


In [5]:
# 简单 softmax
x = tf.placeholder("float", [None, rolldays*factor_nums])
W = tf.Variable(tf.zeros([rolldays*factor_nums,label_nums]))
b = tf.Variable(tf.zeros([label_nums]))
y = tf.nn.softmax(tf.matmul(x,W) + b)
y_ = tf.placeholder("float", [None,label_nums])
# 成本函数, 交叉熵
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
# 变量初始化
init = tf.global_variables_initializer()
# 开始会话
sess = tf.Session()
sess.run(init)

sess.run(train_step,feed_dict={x: xs_train, y_: ys_train})
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
result = sess.run(accuracy, feed_dict={x: xs_test, y_: ys_test})
print(result)
sess.close()

0.327306


In [8]:
# Parameters
learning_rate = 0.001 # 学习速率，
training_iters = 10 # 训练次数
batch_size = 512 # 每次计算数量 批次大小
display_step = 10 # 显示步长

# Network Parameters
n_input = rolldays*factor_nums # 40天×17多因子
n_classes = label_nums # 根据涨跌幅度分成7类别
# 这里注意要使用 one-hot格式，也就是如果分类如3类 -1,0,1 则需要3列来表达这个分类结果，3类是-1 0 1 然后是哪类，哪类那一行为1 否则为0
dropout = 0.8 # Dropout, probability to keep units

# tensorflow 图 Graph 输入 input，这里的占位符均为输入
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

def CNN_Net_three(x,weights,biases,dropout=0.8,m=1):
    
    x = tf.reshape(x, shape=[-1,rolldays,factor_nums,1])
    
    # 卷积层1
    x = tf.nn.conv2d(x, weights['wc1'], strides=[1,m,m,1],padding='SAME')
    x = tf.nn.bias_add(x,biases['bc1'])
    x = tf.nn.relu(x)
    x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
    
    # 卷积层2 
    x = tf.nn.conv2d(x, weights['wc2'], strides=[1,m,m,1],padding='SAME')
    x = tf.nn.bias_add(x,biases['bc2'])
    x = tf.nn.relu(x)
    x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
    
    # 卷积层3 
    x = tf.nn.conv2d(x, weights['wc3'], strides=[1,m,m,1],padding='SAME')
    x = tf.nn.bias_add(x,biases['bc3'])
    x = tf.nn.relu(x)    
    x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
    
    # 全连接层1
    x = tf.reshape(x,[-1,weights['wd1'].get_shape().as_list()[0]])
    x = tf.add(tf.matmul(x,weights['wd1']),biases['bd1'])
    x = tf.nn.relu(x)
    x = tf.nn.dropout(x,dropout)
    
    # 全连接层2
    x = tf.reshape(x,[-1,weights['wd2'].get_shape().as_list()[0]])
    x = tf.add(tf.matmul(x,weights['wd2']),biases['bd2'])
    x = tf.nn.relu(x)
    x = tf.nn.dropout(x,dropout)
    #print (x.get_shape().as_list())
    

    # Output, class prediction
    x = tf.add(tf.matmul(x,weights['out']),biases['out'])
    
    return x

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 16])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 16, 32])),
    'wc3': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([rolldays*factor_nums, batch_size])),
    'wd2': tf.Variable(tf.random_normal([batch_size, 256])),  
    'out': tf.Variable(tf.random_normal([256, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([16])),
    'bc2': tf.Variable(tf.random_normal([32])),
    'bc3': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([batch_size])),
    'bd2': tf.Variable(tf.random_normal([256])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print("cnn setup finished")

cnn setup finished


In [ ]:
# 模型优化
pred = CNN_Net_three(x,weights,biases,dropout=keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred,1),tf.arg_max(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# 更改数据格式，降低均值
saver = tf.train.Saver()
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for step in range(training_iters):
        for i in range(int(len(xs_train)/batch_size)):    
            batch_x = xs_train[i*batch_size:(i+1)*batch_size]
            batch_y = ys_train[i*batch_size:(i+1)*batch_size]
            #print(i,batch_size,batch_x.shape,batch_y.shape)
            sess.run(optimizer,feed_dict={x:batch_x,y:batch_y,keep_prob:dropout})
        loss, acc = sess.run([cost, accuracy], feed_dict={x: xs_test,y: ys_test,keep_prob: 1.})
        print("Iter " ,step , " Minibatch Loss= " , "{:.6f}".format(loss) , ", Training Accuracy= " , "{:.5f}".format(acc))
    save_path = saver.save(sess,'data/test_var.ckpt')
    print ('保持变量')
    print("Optimization Finished!")   
    sess.close()

In [10]:
#预测
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess,'data/test_var.ckpt')
#     trainX_Convolution = sess.run(tmp, feed_dict={x:xs_train, keep_prob:1.})
    # 经过卷积处理的特征向量
    nn_score = sess.run(accuracy,feed_dict={x:xs_train,y: ys_train,keep_prob:1.})
    nn_score1 = sess.run(accuracy,feed_dict={x:xs_test,y: ys_test, keep_prob:1.})
    print(nn_score,'---',nn_score1)
    sess.close()

0.313291 --- 0.327306
